# Pandas

Mit Pandas (= panel data) lassen sich tabellarische Daten sehr komfortabel handhaben. Mehr unter https://pandas.pydata.org/.

Sehr hilfrich:
- https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html
- https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

Pandas nutzt zwei wichtige Datenstrukturen:
- `Series` sind 1-dimensionale Arrays. Im Gegensatz zu Numpy-Arrays haben sie aber einen zusätzlichen Index, über den man komfortabel auf Einträge zugreifen kann. Intern baut dies auf Numpy-Arrays auf.
- `DataFrame` ist eine ganze Tabelle bestehend aus mehreren Series (=Spalten) gleicher Länge.

In [ ]:
import numpy as np
import pandas as pd # empfehlenswerte Konvention

#### DataFrames

In [ ]:
starwars_simple = pd.DataFrame({
    "name"   : ["Luke", "Chewbacca", "Darth Vader"],
    "jedi"   : [True, False, True],
    "height" : [172, 228, 202]})
starwars_simple

Jede Spalte ist eine Series.

In [ ]:
starwars_simple["height"]
type(starwars_simple["height"])

Die vorderste Spalte eines DataFrame (oder Series) ist der sogenannte Index. Er enthält die *Zeilennamen*. Standardmäßig sind dies die Zahlen 0, 1, 2, ...
Man kann jedoch auch einen anderen Index setzen.

In [ ]:
starwars_simple.index
starwars_simple = starwars_simple.set_index("name")
starwars_simple

Man kann eine Series auch explizit erzeugen. Falls man keinen Index angibt, sind es wieder die Zahlen 0, 1, 2, ...

In [ ]:
dummy = pd.Series(np.random.standard_normal(4), index= ["a", "c", "d", "b"]) # spezieller Index
dummy

#### Daten einlesen und schreiben

Pandas bietet eine Vielzahl an Möglichkeiten um Daten aus Datenbanken (via SQL) oder aus Dateien einzulesen, z.B. csv, xlsx, parquet, feather, ...

In [ ]:
starwars = pd.read_csv("starwars.csv") # siehe Dokumentation, ebenso zB pd.read_excel(), pd.read_sql()
starwars

Ersten Einblick in die Daten gewinnen.

In [ ]:
#starwars.head()
starwars.info()
#starwars.describe()
#starwars.shape

Auch das Lesen und Schreiben von xlsx-Dateien ist kein Problem. Hier exemplarisch den Datensatz als xlsx abspreichern. Hierfür muss das Paket *openpyxl* installiert sein.

In [ ]:
starwars.to_excel("starwars.xlsx", sheet_name="some characters", index=False)

#### Auswahl von Spalten oder Zeilen

Wählt man nicht nur einen Spaltennamen sondern eine Liste von mehreren Spalten, so erhält man einen DataFrame.

In [ ]:
starwars[["name", "height"]] 

Bei der Zeilenauswahl möchte man meistens Zeilen selektieren, die bestimmte Eigenschaften erfüllen.
Hierfür schreibt man in die eckigen Klammern `[]` einen True/False-Vektor von der Länge des DataFrames.
Oft sind Vergleichsoperatoren (`<`, `>`, `==`, `<=`, `>=`, `!=`) und Boole'sche Operatoren `&` (and), `|` (or) und `~` (not) sehr hilfreich.

In [ ]:
starwars[starwars["height"] < 180]
starwars["height"] < 180 # dies ergibt einen True/False-Vektor von der Länge des DataFrames

starwars[(starwars["height"] < 180) & (starwars["sex"] == "male")] # Boole'sche Operatoren 

starwars[starwars["eye_color"].isin(["red", "blue"])] # übersichtlicher als (starwars["eye_color"] == "red") | (starwars["eye_color"] == "blue")

starwars[starwars["homeworld"].isna()] # Auswahl von Missing Values mittels .isna() oder .notna()

Möchte man *in einem Schritt* gewisse Zeilen und Spalten auswählen, eignen sich `.loc` und `.iloc`.

In [ ]:
starwars.loc[starwars["height"] < 180, "name"]


Mittels `.loc[rows, columns]` (= location) kann man Einträge über ihren **Zeilenindex** ansprechen.

In [ ]:
starwars = starwars.set_index("name") # sogar mehrere Spalten als Index möglich (sog. MultiIndex)
starwars

In [ ]:
starwars.loc["R2-D2"] # Zeilenauswahl per Zeilenindex
starwars.loc[:, "height"] # alle Zeilen der Spalte
starwars.loc["R2-D2":"Chewbacca", ["mass", "homeworld"]] # Achtung: Slicing inklusive Ende!

Mittels `.iloc[row, column]` (= integer location) kann man Einträge über ihre Zeilen-/Spalten**nummer**, d.h. über die Position und nicht über den Namen, auswählen. (Hier funktioniert das Slicing wie gewohnt.)

In [ ]:
starwars.iloc[0:4, [2, 4]] # Auswahl per integer location
starwars

Mit `.reset_index` lässt sich der Index zu einer normalen Spalte machen. 

In [ ]:
starwars = starwars.reset_index()
starwars

#### Spalten erzeugen, entfernen und umbenennen

Beim berechnen neuer Spalten wird die Mächtigkeit der Vektorisierung deutlich.
Die Berechnungen finden elementweise für jeden Eintrag des Vektors statt.
Eine explizite Schleife ist nicht nötig.

In [ ]:
starwars["BMI"]  = starwars["mass"] / (starwars["height"] / 100)**2
starwars["col1"] = 7 # jeden Eintrag auf diesen skalaren Wert setzen
starwars["col2"] = np.arange(starwars.shape[0]) # die Zahlen 0, 1, 2, ...


starwars = starwars.drop(columns=["col1", "col2"])
starwars

Möchte man komplizierte Berechnungen durchführen, kann man eine Funktion schreiben und diese mittels `map()` oder `apply()` zeilenweise 
anwenden. Mehr dazu später! 

Zur Umbenennung von Spalten übergibt man der Funktion `.rename` ein Dictionary mit Einträgen der Form `{"old_colname": "new_colname"}` oder eine geeignete String-Funktion.

In [ ]:
starwars_renamed = starwars.rename(columns= {"eye_color": "Augenfarbe", "sex": "Geschlecht"})
starwars_renamed
starwars.rename(columns= str.upper).head(2)

#### Visualisierung

(Hier nur ganz kurz. Visualisierung bekommt noch eine Extrasession.)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax = starwars.plot.scatter(x = "height", y= "mass", c = "BMI", colormap='viridis', title = "Star Wars")

#starwars.apply(add_name,axis=1)
for idx, row in starwars.iterrows():
    ax.annotate(row["name"], (row["height"], row["mass"]))

plt.show()

#### Sortieren

In [ ]:
starwars.sort_values(by = "height")
starwars.sort_values(by = "height", ascending= False)
starwars.sort_values(by = ["jedi", "height"], ascending= [True, False])


Die Sortierung hat natürlich Konsequenzen für den Index. 
Möchte man diesen wieder korrigieren, so kann man entweder den Index reseten oder ihn direkt bei der Sortierung ignorieren lassen. 

In [ ]:
starwars.sort_values("height").reset_index(drop = True) # früher war dies gängig
starwars.sort_values("height", ignore_index= True)

#### Aggregation von Daten

Typische Aggregationsfunktionen wie z.B. mean(), sum(), max(), quantile() ... können direkt als Methode von `Series` aufgerufen werden.

In [ ]:
starwars["mass"].mean()
starwars[["height", "mass"]].max()

Sehr oft möchte man derartige Aggregationen gruppenweise, d.h. für jede Ausprägung eines Merkmals, durchführen. Dies ist bekannt als **Split-Apply-Combine**: Die Gesamttabelle wird gemäß der Ausprägungen eines Merkmals in Einzeltabellen geteilt, für jede solche Tabelle berechnet man die Aggregationsfunktion und anschließend werden diese Werte in einer kleineren Tabelle gesammelt.

In [ ]:
starwars[["jedi", "mass"]].groupby("jedi").mean() # eine spezielle Spalte
starwars.groupby("jedi")["mass"].mean() # gleichwertig zu erster Zeile
#starwars.groupby("jedi").mean(numeric_only = True) # für alle! numerischen Spalten

Soeben haben wir direkt nach `.groupby()` eine eckige Klammer mit Spaltennamen genutzt. Dies wird oft gemacht und schränkt die Einzeltabellen auf die genannten Spalten ein. Die Aggregationsfunktion wirkt dann nur noch auf diese Spalten.

Man kann auch nach mehreren Merkmalen gruppieren. Das resultierende Objekt hat dann einen MultiIndex.

In [ ]:
starwars.groupby(["jedi", "sex"])["mass"].mean()
starwars.groupby(["jedi", "sex"]).mean(numeric_only = True)

Sehr oft möchte man für jede Ausprägung eines Merkmals auszählen wie oft es vorkommt. Dies liefert die Grundlage für Balkendiagramme.
(Achtung: Standardmäßig bilden NA-Werte keine eigene Gruppe und werden nicht aufgeführt. Dies ist im Rahmen der Datenaufbereitung allerdings oft relevant. Hierfür kann man in `.groupby()` oder in `.value_counts()` die Option `dropna=False` setzen.)
Da das resultierende Objekt kein DataFrame ist, eignet sich ein `.reset_index()` um den Index zu einer expliziten Spalte zu machen und z.B. Sortierschritte anzuschließen.

In [ ]:
starwars.groupby("sex").size().reset_index()
#starwars["sex"].value_counts()

Es gibt sowohl `.count()` als auch `.size()`. Diese sind sehr ähnlich. Während `.size()` die Zeilenanzahl zurückgibt, liefert `.count()` die Anzahl an Nicht-NA-Werten. Die eine Funktion zählt also NA-Werte mit, die andere hingegen nicht.

#### Datenschubsen für Fortgeschrittene: Long-Format und Wide-Format

<!--- Datenquelle:
- Destatis, Tabelle 12411-0015: Bevölkerung: Kreise, Stichtag (Auswahl: alle Stichtage). (Download Flat-File!)
- Destatis, Tabelle 12411-0018: Bevölkerung: Kreise, Stichtag, Geschlecht, Altersgruppen (Auswahl: alle Stichtage aber nur die 3 Kreise Aschaffenburg, Würzburg und Schweinfurt). (Download Flat-File!) -->

Es gibt verschiedene Möglichkeiten ein und die selben Daten tabellarisch darzustellen.
Je nach Anwendungsfall ist es nötig zwischen diesen Formen zu wechseln. Dies ist manchmal als "pivotieren" bekannt.
Am besten sieht man es an einem Beispiel. 

<!--- Die nachfolgende Tabelle (adaptiert nach Destatis Tabelle 12411-0016) stellt die Bevölkerungsentwicklung der drei Städte Aschaffenburg, Schweinfurt und Würzburg dar.-->

Die nachfolgende Tabelle enthält die Entwicklung der Studierendenzahlen für die TH Aschaffenburg

In [ ]:
th = pd.read_csv("TH-Aschaffenburg.csv")
th.head()

<!--- Zur Einfachheit vernachlässigen wir zunächst die Geschlechtsunterscheidung und beschränken uns auf die Gesamtzahlen.
Die Tabelle ist im Long-Format und enthält für jede Kombination aus Ort und Zeit eine Zeile mit der Bevölkerungszahl.
Dies hat den Vorteil, dass problemlos weitere Orte und Zeiten ergänzt werden können ohne die Struktur der Tabelle ändern zu müssen.-->

Zur Einfachheit vernachlässigen wir zunächst die Gesamtstudierendenzahlen und beschränken uns nur auf die Studiengänge *BW* (Betriebswirtschaft), *SD* (Software Design) und *EIT* (Elektro- und Informationstechnik) seit dem Wintersemester 2020.

In [ ]:
th_small = th[th["Fach"].isin(["BW", "SD", "EIT"])  & th["Jahr"].isin(["WS 20/21", "WS 21/22", "WS 22/23"])].drop(columns = "Studierende").reset_index(drop = True)
th_small

Die obige Tabelle beinhaltet für jede Kombination von Studienjahr und Fach eine Zeile. 
Dies nennt man **Long-Format**.
Insbesondere für Zeitreihen ist es jedoch oft übersichtlicher die Daten anders anzuordnen.
Hier wollen wir die Anfängerzahlen jedes Faches in einer separaten Spalte darstellen.
Dies ist dann das **Wide-Format**.

Mit dem Befehl `pivot()` kann man die Felder geeignet "rotieren".
Hierbei muss man angeben welche Spalten als Index erhalten bleiben sollen, welche Spalte die neuen Spaltennamen enthält und welches die eigentlichen Werte sind.
Das Wide-Format mag übersichtlicher erscheinen, hat jedoch den Nachteil, dass man das Tabellenschema ändern muss wenn neue Studiengänge hinzukommen.

In [ ]:
th_small_wide = th_small.pivot(index="Jahr", columns="Fach", values = "Anfänger")
th_small_wide

Natürlich kann man auch vom Wide-Format zum Long-Format konvertieren.
Dies funktionert mit `melt()`.
(Zunächst machen wir jedoch mit `reset_index()` den Index zur einer regulären Spalte.)
Im Argument `id_vars` listet man alle Spalten, die konstant gehalten werden sollen - alle anderen Spalten werden zu zwei neuen Spalten "pivotiert", indem jede Kombination aus Spaltenname und jeweilem Eintrag eine neue Zeile bilden. 

In [ ]:
th_small_wide.reset_index().melt(id_vars = "Jahr")

Soeben hatten wir den Datensatz noch auf die Anfängerzahlen eingeschränkt und die Spalte *Studierende* entfernt.
So haben wir `pivot()` nur die Spalte *Anfänger* pivotieren lassen.

Belässt man hingegen die Studienrendenzahlen im Datensatz, so können wir auch diese mitpivotieren.
Hierbei entsteht für die Spalten ein **MultiIndex**.

In [ ]:
th_small = th[th["Fach"].isin(["BW", "SD", "EIT"])  & th["Jahr"].isin(["WS 20/21", "WS 21/22", "WS 22/23"])].reset_index(drop = True)
th_small

In [ ]:
th_small_wide = th_small.pivot(index= ["Jahr"], columns= "Fach", values= ["Anfänger", "Studierende"])
th_small_wide

In [ ]:
th_small_wide.columns
th_small_wide["Anfänger"]
th_small_wide[("Anfänger", "EIT")]

Eng verwandt mit `pivot()` und `melt()` sind die Funktionen `stack()` und `unstack()`.
Sie sind vor allem im Zusammenspiel mit Indexen und MultiIndexen sinnvoll.

- `stack()` pivotiert ("stapelt") alle Spaltenlabel (außer dem Index) und liefert eine Series (oder einen DataFrame) mit einem zusätzlichen inneren Indexlevel.
- `unstack()` pivotiert das innerste Indexlevel zu Spaltenlabeln und erzeugt so einen DataFrame.

Beschränken wir uns zur Übersichtlichkeit auf die obigen Anfängerzahlen.

In [ ]:
beginners = th_small_wide["Anfänger"]
beginners

`stack()` rotiert alle Spaltennamen zu einem neuen inneren Indexlevel und liefert somit eine Series mit einem zweistufigen Multiindex.

In [ ]:
beginners_stacked = beginners.stack()
beginners_stacked

Mittels `unstack()` wird das innerste Level eines MultiIndex zu neuen Spaltennamen und wir erhalten wieder den ursprünglichen DataFrame.
(Möchte man anstatt des innersten Levels ein anderes Indexlevel pivotieren, so kann man sowohl `stack` als auch `unstack` ein Indexlevel angeben.)

In [ ]:
beginners_stacked.unstack()

Nebenbei: Anstatt `pivot` aufzurufen kann man auch mittels `set_index` einen MultiIndex erzeugen und im Anschluss `unstack` aufrufen.

#### Zusammenführen mehrerer Tabellen

Sehr oft möchte man mehrere Tabellen kombinieren um Informationen anzureichern.
Auch mit Pandas kann man mehrere DataFrames verjoinen, so wie man es von SQL kennt.

Die einfachste Variante um zwei DataFrames zu kombinieren ist das simple Untereinanderhängen.
Dies ergibt selbstverständlich nur dann Sinn, wenn die Spalten übereinstimmen (oder zumindest Teilmengen voneinander sind).
Betrachten wir hier beispielsweise die Daten zum Fach *MEDS* sowie zu *BW KMU*.
Mit `concat()` lassen sich die beiden Tabellen direkt hintereinanderhängen.

In [ ]:
df_1 = th[th["Fach"] == "MEDS"]
df_2 = th[th["Fach"] == "BW KMU"]

pd.concat([df_1, df_2])

Inhaltlich spannender ist das Verjoinen von Tabellen um zusätzliche Spalten zu erhalten.
In der bisherigen Tabelle sind z.B. nur die Studiengangskürzel enthalten und auch eine Zuordnung zu Fakultäten fehlt.
Diese Information liefert die folgende Tabelle

In [ ]:
th_names = pd.read_csv("TH-Aschaffenburg_Faecher.csv")
th_names.head()

Mittels `merge()` kann man in Pandas alle Arten von JOIN-Operationen durchführen, wie man sie aus SQL kennt.
Hier wollen wir die Tabelle *th_small* mit den jeweiligen Studiengangsinformationen anreichern.

In [ ]:
pd.merge(left=th_small, right = th_names, how = "left", left_on = "Fach", right_on = "Abkürzung")

Mit `merge()` lassen sich auch andere Joins durchführen, beispielsweise ein RIGHT JOIN (aber auch INNER JOIN oder OUTER JOIN).

In [ ]:
pd.merge(left = th_small, right = th_names, how = "right", left_on = "Fach", right_on = "Abkürzung")